In [ ]:
import os
import re
import datetime
import tempfile
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", None)

In [ ]:
start_year=2023
end_year=2023
start_month=8
end_month=8
REV_CSV="./data/account-statement_2023-08-01_2023-08-21.csv"
UNI_CSV="./data/Elenco_Movimenti_Luglio.csv"
start_date = pd.Timestamp(year=start_year, month=start_month, day=1)
end_date = pd.Timestamp(year=end_year, month=end_month, day=1) + pd.offsets.MonthEnd(0)

In [ ]:
def to_float(number_str):
  """
  1.950,35 => 1950.35
  """
  return float(number_str.replace(".","").replace(",","."))

In [ ]:
def parse_revolut(csv_file, remove_positive_cash_flow=False, remove_negative_cash_flow=False):
    
    df = pd.read_csv(
        csv_file,
        sep=",",
        parse_dates=['Started Date', 'Completed Date'],
        date_format='%Y-%m-%d %H:%M:%S'
    )
    print(f"Loaded {len(df)} records")
    df = df[(df['Started Date'] >= start_date) & (df['Completed Date'] <= end_date)]
    print(f"Removed records (timestamps). Remaining: {len(df)}")
    df.drop(df[df.Description == "Top-Up by *9994"].index, inplace=True)
    df.drop(df[df.Description == "Payment from Baroncelli Leonardo"].index, inplace=True)
    df.drop(df[df.Description == "To EUR Risparmi"].index, inplace=True)
    df.drop(df[df.Description == "To EUR"].index, inplace=True)
    df.drop(df[df.Description == "To Leonardo Baroncelli"].index, inplace=True)

    print(f"Removing records (in cash-flow: top-ups from Unicredit, transfers to savings, transfer to Unicredit). Remaining: {len(df)}")

    # Removing records with 0 amount, if any
    df.drop(df[df['Amount'] == 0].index, inplace=True)

    # Remove earnings
    if remove_positive_cash_flow:
        df.drop(df[df['Amount'] > 0].index, inplace=True)
        print(f"Removing records (in cash-flow). Remaining: {len(df)}")

    if remove_negative_cash_flow:
        df.drop(df[df['Amount'] < 0].index, inplace=True)
        print(f"Removing records (out cash-flow). Remaining: {len(df)}")
    
    # Remove atm withdrawal
    df.drop(df[df['Type'] == "ATM"].index, inplace=True)
    return df

In [ ]:
def parse_unicredit(csv_file, remove_positive_cash_flow=False, remove_negative_cash_flow=False):
    
    df = pd.read_csv(
        csv_file,
        sep=";",
        parse_dates=['Data Registrazione', 'Data valuta'],
        date_format='%d.%m.%Y'
    )
    print(f"Loaded {len(df)} records")
    df.rename(columns={
        'Data Registrazione': 'Started Date', 
        'Data valuta': 'Completed Date',
        'Descrizione' : 'Description',
        'Importo (EUR)' : 'Amount'
    }, inplace=True)
    df = df[(df['Started Date'] >= start_date) & (df['Completed Date'] <= end_date)]
    print(f"Removing records (timestamps). Remaining: {len(df)}")
    for col_name in df.columns.values:
        if "Unnamed" in col_name:
            df.drop(col_name, inplace=True, axis=1)
    assert len(df.columns) == 4
    df["Amount"] = df["Amount"].transform(to_float)

    # Removing records with 0 amount, if any
    df.drop(df[df['Amount'] == 0].index, inplace=True)
    print(f"Removing records (amount=0). Remaining: {len(df)}")
    
    df = df[~df['Description'].str.contains("Revolut")]
    df = df[~df['Description'].str.contains("LEONARDO BARONCELLI REV")]
    print(f"Removing records (transfers to Revolut). Remaining: {len(df)}")

    df = df[~df['Description'].str.contains("PRELIEVO MASTERCARD")]
    print(f"Removing records (cash out). Remaining: {len(df)}")

    if remove_positive_cash_flow:
        df.drop(df[df['Amount'] > 0].index, inplace=True)
        print(f"Removing records (in cash-flow). Remaining: {len(df)}")

    if remove_negative_cash_flow:
        df.drop(df[df['Amount'] < 0].index, inplace=True)
        print(f"Removing records (out cash-flow). Remaining: {len(df)}")    
    
    return df

In [ ]:
def get_raw(df, change_sign_of_amounts=False):
    if change_sign_of_amounts:
        df['Amount'] = df['Amount'].apply(lambda x: x*-1) 
    df['Description'] = df['Description'].apply(lambda x: " ".join(x.split()))
    pattern = r'PAGAMENTO MASTERCARD.*?CARTA \*\d{4} DI EUR \d+,\d+ '
    df['Description'] = df['Description'].apply(lambda x: re.sub(pattern,'', x))
    with tempfile.TemporaryDirectory() as tmp:
        path = os.path.join(tmp, 'temp_csv')
        df.to_csv(path, columns=["Description", "Started Date", "Amount"], sep=';', index=False, header=False)
        with open(path, "r") as f:
            lines = f.readlines()
    for l in lines:
        print(l, end='')

# Negative cash-flow

In [ ]:
rev_df = parse_revolut(
    REV_CSV,
    remove_positive_cash_flow=True
)

In [ ]:
get_raw(rev_df, change_sign_of_amounts=True)

In [ ]:
rev_df

In [ ]:
uni_df = parse_unicredit(
    UNI_CSV,
    remove_positive_cash_flow=True
)

In [ ]:
#display(uni_df)

In [ ]:
get_raw(uni_df, change_sign_of_amounts=True)

# Positive cash-flow

In [ ]:
rev_df = parse_revolut(
    REV_CSV,
    remove_negative_cash_flow=True
)

In [ ]:
rev_df

In [ ]:
uni_df = parse_unicredit(
    UNI_CSV,
    remove_negative_cash_flow=True
)

In [ ]:
uni_df